In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from keras.utils import plot_model
from keras.layers import Dense,Conv2D,Flatten,BatchNormalization,Dropout,MaxPooling2D,GlobalAvgPool2D
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from keras import Sequential
import matplotlib.pyplot as plt
np.random.seed = 32

2023-06-02 23:49:58.642695: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 23:49:58.681643: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 23:49:59.266214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
image_dir = "./pizza_not_pizza/"

In [3]:
not_pizza = [(os.path.join(image_dir,"not_pizza",image),0) for image in os.listdir(os.path.join(image_dir,"not_pizza")) if image.split(".")[1] == "jpg"]
pizza = [(os.path.join(image_dir,"pizza",image),1) for image in os.listdir(os.path.join(image_dir,"pizza")) if image.split(".")[1] == "jpg"]

df = pd.DataFrame(not_pizza+pizza,columns=['filename','category'])
df

,filename,category
0,./pizza_not_pizza/not_pizza/2712131.jpg,0
1,./pizza_not_pizza/not_pizza/3610311.jpg,0
2,./pizza_not_pizza/not_pizza/1118613.jpg,0
3,./pizza_not_pizza/not_pizza/2084262.jpg,0
4,./pizza_not_pizza/not_pizza/2118551.jpg,0
...,...,...
1961,./pizza_not_pizza/pizza/1088332.jpg,1
1962,./pizza_not_pizza/pizza/2250611.jpg,1
1963,./pizza_not_pizza/pizza/796922.jpg,1
1964,./pizza_not_pizza/pizza/1593835.jpg,1


In [4]:
import cv2
img = cv2.imread(df['filename'][100])
cv2.imshow('path',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
df[df['category']==0].count()

filename    983
category    983
dtype: int64

In [6]:
df[df['category']==1].count()

filename    983
category    983
dtype: int64

In [7]:
df= df.sample(frac = 1)
df=df.reset_index()
df

,index,filename,category
0,1561,./pizza_not_pizza/pizza/724290.jpg,1
1,1440,./pizza_not_pizza/pizza/203831.jpg,1
2,1250,./pizza_not_pizza/pizza/3443136.jpg,1
3,1818,./pizza_not_pizza/pizza/3185774.jpg,1
4,1270,./pizza_not_pizza/pizza/3478964.jpg,1
...,...,...,...
1961,716,./pizza_not_pizza/not_pizza/3290376.jpg,0
1962,158,./pizza_not_pizza/not_pizza/3372205.jpg,0
1963,761,./pizza_not_pizza/not_pizza/2527642.jpg,0
1964,415,./pizza_not_pizza/not_pizza/2848506.jpg,0


In [13]:
df = df.drop(columns='index')

In [14]:
df

,filename,category
0,./pizza_not_pizza/pizza/724290.jpg,1
1,./pizza_not_pizza/pizza/203831.jpg,1
2,./pizza_not_pizza/pizza/3443136.jpg,1
3,./pizza_not_pizza/pizza/3185774.jpg,1
4,./pizza_not_pizza/pizza/3478964.jpg,1
...,...,...
1961,./pizza_not_pizza/not_pizza/3290376.jpg,0
1962,./pizza_not_pizza/not_pizza/3372205.jpg,0
1963,./pizza_not_pizza/not_pizza/2527642.jpg,0
1964,./pizza_not_pizza/not_pizza/2848506.jpg,0
